<a href="https://colab.research.google.com/github/hamojb/AD_research/blob/main/colab_playaround.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from nibabel.testing import data_path
import tensorflow as tf
import nibabel as nib
import os, psutil
from collections import Counter
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

3634888704


In [53]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [54]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7805169678264949134, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14509932544
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13072783316174918671
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

# Define Helper Functions

In [55]:
def show_slices(slices, col = None):
  """ Function to display row of image slices """
  fig, axes = plt.subplots(1, len(slices))
  for i, slice in enumerate(slices):
      axes[i].imshow(slice.T, cmap=col, origin="lower")
  return

def read_data(data_path: str): 
  """ Function to read in data of each brain scan """
  temp_path = os.path.join(data_path)
  temp_img = nib.load(temp_path)
  temp_data = temp_img.get_fdata()
  return temp_data



def produce_image(data_path: str): 
  """ Function to read in data of each brain scan """
  temp_path = os.path.join(data_path)
  temp_img = nib.load(temp_path)
  return temp_img

# def populate_images(oasis_dir):#, df):
#   """ Function to populate data """ 
#   data = []
#   images = []
#   filenames = []
#   for filename in os.listdir(oasis_dir):
#     print(filename)
#     # if file_str in df
#       # new_data, img = read_data(oasis_dir + filename)
#       # data.append(new_data)
#       # images.append(img)
#   return data, images, filenames

def populate_images(oasis_dir, train_val_test):
  """ Function to populate data """ 
  ret_df = {}
  for filename in os.listdir(oasis_dir):
    new_str = filename.split(".")[0].split("_")[0]
    scan = filename.split(".")[0]
    if new_str in train_val_test:
      new_data = read_data(oasis_dir + filename)
      ret_df[scan] = new_data
    else: 
      continue
    
  return ret_df

def match_data(scan_data, diagnosis_data): 
  empty_df = pd.DataFrame()
  scan_patients = list(set(scan_data.Subject.values))
  for patient in scan_patients:
    temp_scans = scan_data[scan_data['Subject'] == patient]
    temp_scans = temp_scans.sort_values('ID')
    temp_diag = diagnosis_data[diagnosis_data['Subject'] == patient]
    temp_diag = temp_diag.sort_values('ID')
    test = pd.merge_asof(temp_scans, temp_diag, on="ID", allow_exact_matches=True, direction = "forward")
    empty_df = empty_df.append(test)
  empty_df = empty_df.drop(columns = ['Inserted_x', 'Inserted_y', 'Subject_y', 'ageAtEntry'])
  empty_df = empty_df.rename(columns = {"Subject_x":"Subject","FS_FSDATA ID":"MRI_ID"})
  empty_df = empty_df.reset_index().drop(columns = ["index"])
  empty_df = empty_df.dropna()
  return empty_df

Quick demonstation on how to read the data in using the `nibabel` package

In [56]:
# path = os.path.join("/content/drive/MyDrive/oasis_original/OAS30001_Freesurfer53_d0129.mgz.nii.gz")
# img = nib.load(path)
# data = img.get_fdata()

In [57]:
# slice_0 = data[128, :, :]
# slice_1 = data[:, 128, :]
# slice_2 = data[:, :, 128]
# show_slices([slice_0, slice_1, slice_2], "gray")

Define functions to read all data in from `oasis_originals`. 

# Read data and store in variable. 

In [58]:
clinical_data = pd.read_csv("/content/drive/MyDrive/extra_data/clinical_data.csv")
clinical_data['ID'] = clinical_data['ADRC_ADRCCLINICALDATA ID'].str.split('_ClinicalData_', 1, expand=True)[1]
clinical_data = clinical_data.drop(columns = ['ADRC_ADRCCLINICALDATA ID', 'Date', 'Subject.1', 'id'])
clinical_data['Inserted'] = pd.to_datetime(clinical_data['Inserted'])
clinical_data['ID'] = clinical_data['ID'].str.extract('(\d+)', expand=False).astype(int)

In [59]:
scan_info = pd.read_csv("/content/drive/MyDrive/extra_data/freesurfers_mri_info.csv")
scan_info['ID'] = scan_info['FS_FSDATA ID'].str.split('_Freesurfer', 1, expand = True)[1]
scan_info = scan_info.drop(columns = ['Session', 'FS Date'])
scan_info.ID = scan_info.ID.str[3:]
scan_info['Inserted'] = pd.to_datetime(scan_info['Inserted'])
scan_info['ID'] = scan_info['ID'].str.extract('(\d+)', expand=False).astype(int)

In [60]:
scan_and_diag = match_data(scan_info, clinical_data)
scans_sorted = scan_and_diag.sort_values(["Subject", "ID"]).reset_index().drop(columns = ["index", "ID"])
scans_sorted

,MRI_ID,Subject,cdr
0,OAS30001_Freesurfer53_d0129,OAS30001,0.0
1,OAS30001_Freesurfer53_d0757,OAS30001,0.0
2,OAS30001_Freesurfer53_d2430,OAS30001,0.0
3,OAS30001_Freesurfer53_d3132,OAS30001,0.0
4,OAS30002_Freesurfer53_d0653,OAS30002,0.0
...,...,...,...
1824,OAS31168_Freesurfer53_d2526,OAS31168,0.0
1825,OAS31169_Freesurfer53_d0620,OAS31169,0.5
1826,OAS31170_Freesurfer53_d2410,OAS31170,1.0
1827,OAS31172_Freesurfer53_d0407,OAS31172,0.0


In [61]:
ALTERNATIVE_CASE = True
if ALTERNATIVE_CASE: 
  scans_sorted['cdr'] = scans_sorted['cdr'].astype(str).replace({'0.0':'HC', '0.5': 'MCI', '1.0': 'MCI', '2.0': 'AD', '3.0': 'AD'})
scans_sorted

,MRI_ID,Subject,cdr
0,OAS30001_Freesurfer53_d0129,OAS30001,HC
1,OAS30001_Freesurfer53_d0757,OAS30001,HC
2,OAS30001_Freesurfer53_d2430,OAS30001,HC
3,OAS30001_Freesurfer53_d3132,OAS30001,HC
4,OAS30002_Freesurfer53_d0653,OAS30002,HC
...,...,...,...
1824,OAS31168_Freesurfer53_d2526,OAS31168,HC
1825,OAS31169_Freesurfer53_d0620,OAS31169,MCI
1826,OAS31170_Freesurfer53_d2410,OAS31170,MCI
1827,OAS31172_Freesurfer53_d0407,OAS31172,HC


In [62]:
Counter(scans_sorted['cdr'])

Counter({'AD': 20, 'HC': 1460, 'MCI': 349})

We will now check to make sure that in our directory of images we have the correct MRI_IDs to link to the existing data. 

In [63]:
num = 0
for i in os.listdir(r'/content/drive/MyDrive/oasis_original/'):
  if i.split(".")[0] in scans_sorted.MRI_ID.values: 
    num += 1
print(num)

1824


In [64]:
subj_ids = list(set(scans_sorted["Subject"]))
inv_test = int(0.15*len(subj_ids))
print(inv_test)

140


In [65]:
from random import shuffle
shuffle(subj_ids)
train_val = subj_ids[inv_test:]
train = train_val[inv_test:]
val = train_val[:inv_test]
test = subj_ids[:inv_test]

In [66]:
# len(train)/8

In [67]:
len(val)

140

In [68]:
len(test)

140

In [69]:
overlaps = set(train).intersection(val, test)
if len(overlaps) == 0: 
  print("CLEAR: No overlaps")
else: 
  print("ERROR: Overlaps exist")
  print(overlaps)

CLEAR: No overlaps


In [70]:
training = scans_sorted[scans_sorted['Subject'].isin(train)]
training.head()

,MRI_ID,Subject,cdr
0,OAS30001_Freesurfer53_d0129,OAS30001,HC
1,OAS30001_Freesurfer53_d0757,OAS30001,HC
2,OAS30001_Freesurfer53_d2430,OAS30001,HC
3,OAS30001_Freesurfer53_d3132,OAS30001,HC
4,OAS30002_Freesurfer53_d0653,OAS30002,HC


In [71]:
training[training.cdr == 'AD']

,MRI_ID,Subject,cdr
166,OAS30098_Freesurfer53_d0036,OAS30098,AD
257,OAS30145_Freesurfer53_d4247,OAS30145,AD
274,OAS30158_Freesurfer53_d2467,OAS30158,AD
396,OAS30244_Freesurfer53_d1526,OAS30244,AD
609,OAS30373_Freesurfer53_d1211,OAS30373,AD
660,OAS30408_Freesurfer50_d1193,OAS30408,AD
950,OAS30616_Freesurfer53_d0199,OAS30616,AD
1162,OAS30753_Freesurfer53_d0035,OAS30753,AD
1322,OAS30853_Freesurfer53_d0100,OAS30853,AD
1472,OAS30948_Freesurfer53_d0076,OAS30948,AD


In [72]:
validation = scans_sorted[scans_sorted['Subject'].isin(val)]
validation.head()

,MRI_ID,Subject,cdr
17,OAS30005_Freesurfer51_d0143,OAS30005,HC
18,OAS30005_Freesurfer53_d0581,OAS30005,HC
19,OAS30005_Freesurfer53_d1274,OAS30005,HC
20,OAS30005_Freesurfer53_d2377,OAS30005,HC
21,OAS30005_Freesurfer53_d2384,OAS30005,HC


In [73]:
validation[validation.cdr == 'AD']

,MRI_ID,Subject,cdr
1336,OAS30860_Freesurfer50_d0638,OAS30860,AD
1543,OAS30994_Freesurfer53_d0070,OAS30994,AD


In [74]:
testing = scans_sorted[scans_sorted['Subject'].isin(test)]
testing.head()

,MRI_ID,Subject,cdr
33,OAS30009_Freesurfer53_d0148,OAS30009,HC
34,OAS30009_Freesurfer53_d1210,OAS30009,HC
39,OAS30013_Freesurfer53_d0102,OAS30013,HC
63,OAS30031_Freesurfer53_d0427,OAS30031,AD
67,OAS30033_Freesurfer53_d0133,OAS30033,HC


In [75]:
testing[testing.cdr == 'AD']

,MRI_ID,Subject,cdr
63,OAS30031_Freesurfer53_d0427,OAS30031,AD
198,OAS30114_Freesurfer53_d0086,OAS30114,AD
1261,OAS30811_Freesurfer53_d7690,OAS30811,AD
1342,OAS30865_Freesurfer53_d0109,OAS30865,AD
1536,OAS30987_Freesurfer53_d0965,OAS30987,AD
1547,OAS30999_Freesurfer53_d0153,OAS30999,AD
1563,OAS31008_Freesurfer53_d0142,OAS31008,AD


In [76]:
Counter(training['cdr'])

Counter({'AD': 11, 'HC': 1042, 'MCI': 250})

In [77]:
Counter(validation['cdr'])

Counter({'AD': 2, 'HC': 200, 'MCI': 54})

In [78]:
Counter(testing['cdr'])

Counter({'AD': 7, 'HC': 218, 'MCI': 45})

Good enough... for now 

In [79]:
# val1 = val[:int(len(val)/2)]
# val2 = val[int(len(val)/2):]

In [80]:
train1 = train[:10]

In [81]:
directory = r'/content/drive/MyDrive/oasis_original/'
train_data1 = populate_images(directory, train1)
# val_data = populate_images(directory, val)

# test_data = populate_images(directory, test)
# train_data = populate_images(directory, train)


In [82]:
df = pd.DataFrame(list(train_data1.items()),columns = ['MRI_ID','Data'])

In [83]:
new_one = training.merge(df, how = "left", on = "MRI_ID")

In [84]:
new_one

,MRI_ID,Subject,cdr,Data
0,OAS30001_Freesurfer53_d0129,OAS30001,HC,NaN
1,OAS30001_Freesurfer53_d0757,OAS30001,HC,NaN
2,OAS30001_Freesurfer53_d2430,OAS30001,HC,NaN
3,OAS30001_Freesurfer53_d3132,OAS30001,HC,NaN
4,OAS30002_Freesurfer53_d0653,OAS30002,HC,NaN
...,...,...,...,...
1298,OAS31168_Freesurfer53_d0148,OAS31168,HC,NaN
1299,OAS31168_Freesurfer53_d1566,OAS31168,HC,NaN
1300,OAS31168_Freesurfer53_d2526,OAS31168,HC,NaN
1301,OAS31169_Freesurfer53_d0620,OAS31169,MCI,NaN


In [85]:
playaround_data = new_one[new_one['Data'].notnull()]

In [86]:
len(playaround_data)

27

In [87]:
y = playaround_data['cdr']
x = playaround_data['Data']

In [87]:
from tensorflow import keras
model = keras.models.load_model('path/to/location')